# Part 2: Deep Dive - 문헌 검색 (Agent Laboratory / PaSa)

**SNU AI Psychology Workshop - February 2026**

---

## 학습 목표
1. Agent Laboratory와 PaSa의 코드 구조 이해
2. arXiv/Scholar 자동 검색 실행
3. 검색 로직 커스텀 (쿼리, 필터, 정렬)

---
## Setup

In [ ]:
# Cell 1: 환경 설정 (Colab/로컬 자동 감지)
import os
import sys

# Colab 환경 감지
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    WORKSHOP_DIR = "/content/drive/MyDrive/aiworkshop_Feb2026/"
    os.makedirs(WORKSHOP_DIR, exist_ok=True)
    os.chdir(WORKSHOP_DIR)
    IN_COLAB = True
    print("🌐 Colab 환경에서 실행 중")
except ImportError:
    # 로컬 환경: 프로젝트 루트로 이동
    current_dir = os.getcwd()
    if current_dir.endswith('notebooks'):
        os.chdir('..')
    WORKSHOP_DIR = os.getcwd()
    IN_COLAB = False
    print("💻 로컬 환경에서 실행 중")

print(f"작업 폴더: {WORKSHOP_DIR}")

In [ ]:
# Cell 2: 패키지 설치 + 저장소 클론
# Colab: 자동 설치
# 로컬: pyproject.toml로 미리 설치 필요 (uv pip install -e . 또는 pip install -e .)

if IN_COLAB:
    print("📦 패키지 설치 중...")
    %pip install arxiv openai python-dotenv pandas -q
    
    # Agent Laboratory 클론
    !git clone https://github.com/SamuelSchmidgall/AgentLaboratory.git 2>/dev/null || echo "  ✓ AgentLaboratory already cloned"
    
    # PaSa 클론  
    !git clone https://github.com/bytedance/pasa.git 2>/dev/null || echo "  ✓ PaSa already cloned"
    
    print("✅ 설치 완료!")
else:
    print("✅ 로컬 환경: pyproject.toml로 설치된 패키지 사용")
    print("   (미설치 시: uv pip install -e . 또는 pip install -e .)")
    
    # 로컬에서 저장소 클론 (WORKSHOP_DIR 기준, 있으면 건너뜀)
    import subprocess
    repos = [
        ("AgentLaboratory", "https://github.com/SamuelSchmidgall/AgentLaboratory.git"),
        ("pasa", "https://github.com/bytedance/pasa.git")
    ]
    
    for name, url in repos:
        repo_path = os.path.join(WORKSHOP_DIR, name)
        if not os.path.exists(repo_path):
            print(f"   📥 {name} 클론 중...")
            subprocess.run(["git", "clone", url, repo_path], capture_output=True)
            print(f"   ✅ {name} 클론 완료")
        else:
            print(f"   ✓ {name} already exists")

In [ ]:
# Cell 3: API Key 로딩
import os
from pathlib import Path

# Colab 환경 체크
if IN_COLAB:
    # Colab userdata에서 키 불러오기
    try:
        from google.colab import userdata
        OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
        GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
        print("✅ Colab Secrets에서 API Key 로딩 완료")
    except:
        print("⚠️ Colab Secrets에 API KEY를 추가하세요")
        OPENAI_API_KEY = None
        GEMINI_API_KEY = None
else:
    # 로컬 환경: dotenv 사용
    try:
        from dotenv import load_dotenv
        load_dotenv()
        OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
        GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
        
        if OPENAI_API_KEY or GEMINI_API_KEY:
            print("✅ .env 파일에서 API Key 로딩 완료")
        else:
            print("⚠️ .env 파일에 API KEY를 추가하세요")
    except ImportError:
        print("⚠️ python-dotenv가 설치되지 않았습니다")
        OPENAI_API_KEY = None
        GEMINI_API_KEY = None

# 환경변수 설정
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY or ''
os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY or ''

# 확인
print(f"\n🔑 OpenAI: {'설정됨' if OPENAI_API_KEY else '없음 (LLM 기능 제한)'}")
print(f"🔑 Gemini: {'설정됨' if GEMINI_API_KEY else '없음 (LLM 기능 제한)'}")
print("💡 arXiv 검색은 API key 없이도 사용 가능합니다")

---
## 1. 파악하기: Agent Laboratory 코드 구조

### Agent Laboratory 구조

```
AgentLaboratory/
├── agents/
│   └── literature_agent.py   # 문헌 검색 에이전트 ⭐
├── tools/
│   └── arxiv_search.py       # arXiv API 래퍼 ⭐
└── run.py                    # 메인 실행
```
💡 실험 설계/보고서 기능도 있지만, 워크샵에서는 **문헌 검색**만 사용

**핵심 흐름:**
1. 연구 주제 입력
2. `literature_agent`가 arXiv에서 관련 논문 검색
3. 논문 요약 및 관련성 평가

In [ ]:
# Cell 6: Agent Laboratory 핵심 모듈 확인
import os

# 폴더 구조 확인
agent_lab_path = "AgentLaboratory"
if os.path.exists(agent_lab_path):
    print("📁 AgentLaboratory 구조:")
    for root, dirs, files in os.walk(agent_lab_path):
        level = root.replace(agent_lab_path, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files[:5]:  # 처음 5개 파일만
            if file.endswith('.py'):
                print(f"{subindent}{file}")
else:
    print("AgentLaboratory가 클론되지 않았습니다. Cell 2를 실행하세요.")

In [ ]:
# Cell 7: arXiv 검색 모듈 직접 살펴보기
import arxiv

# arXiv 검색 함수 (Agent Laboratory 스타일)
def search_arxiv_papers(query, max_results=10, sort_by="relevance"):
    """
    Agent Laboratory의 arxiv_search.py 핵심 로직
    """
    sort_criterion = {
        "relevance": arxiv.SortCriterion.Relevance,
        "date": arxiv.SortCriterion.SubmittedDate,
    }.get(sort_by, arxiv.SortCriterion.Relevance)
    
    client = arxiv.Client()
    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=sort_criterion
    )
    
    papers = []
    for result in client.results(search):
        papers.append({
            "title": result.title,
            "authors": [a.name for a in result.authors],
            "abstract": result.summary,
            "published": result.published.strftime("%Y-%m-%d"),
            "pdf_url": result.pdf_url,
            "arxiv_id": result.entry_id.split('/')[-1]
        })
    return papers

print("search_arxiv_papers 함수 정의 완료")
print(f"시그니처: search_arxiv_papers(query, max_results=10, sort_by='relevance')")

### PaSa (Paper Search Agent) 구조

```
pasa/
├── agents/
│   ├── crawler.py     # 논문 크롤링 에이전트 ⭐
│   └── selector.py    # 관련성 평가 (PPO 학습됨) ⭐
├── search/
│   ├── arxiv.py       # arXiv 검색
│   └── scholar.py     # Google Scholar 검색
└── run_search.py
```

**핵심 차이점:**
- PaSa는 **PPO로 학습된 Selector**가 관련성 평가
- Agent Laboratory보다 더 정교한 필터링

### 질문

두 도구의 구조를 비교해보세요:
- Agent Laboratory: 문헌 검색 자동화 (워크샵에서는 이 부분만 사용)
- PaSa: 검색 특화 (PPO 기반 관련성 평가)

본인 연구에는 어떤 접근이 더 맞을까요?

---
## 2. 써보기: arXiv 논문 검색

### 샘플 데이터

테스트할 연구 주제:
- "LLM reasoning and cognitive psychology"
- "AI agent for scientific discovery"

In [ ]:
# Cell 10: 샘플 검색 실행 (발표자 데모)
sample_query = "LLM reasoning cognitive psychology"

print(f"🔍 검색어: {sample_query}\n")
results = search_arxiv_papers(sample_query, max_results=5)

print(f"📚 검색 결과: {len(results)}개 논문\n")
for i, paper in enumerate(results, 1):
    print(f"{i}. {paper['title']}")
    print(f"   저자: {', '.join(paper['authors'][:3])}...")
    print(f"   날짜: {paper['published']}")
    print(f"   ID: {paper['arxiv_id']}")
    print()

In [ ]:
# Cell 11: 결과 시각화
import pandas as pd

# DataFrame으로 변환
df = pd.DataFrame(results)
df['abstract_preview'] = df['abstract'].str[:100] + '...'
df['author_count'] = df['authors'].apply(len)

print("📊 검색 결과 요약")
display(df[['title', 'published', 'author_count']].head())

### DIY: 본인 연구 주제로 검색해보세요

In [ ]:
# DIY Cell 13: 본인 연구 주제 검색
# 힌트: my_query 변수만 바꾸면 됩니다

my_query = "your research topic here"  # <- 이 부분 수정!

my_results = search_arxiv_papers(my_query, max_results=10)

print(f"🔍 검색어: {my_query}")
print(f"📚 결과: {len(my_results)}개 논문\n")

for i, paper in enumerate(my_results, 1):
    print(f"{i}. {paper['title']}")
    print(f"   {paper['published']} | {paper['arxiv_id']}")
    print()

In [ ]:
# DIY Cell 14: 검색 결과 분석
# 질문: 검색 결과 중 관련성 높은 논문은 몇 개인가요?

# 관련성 평가 (1-5점)
# 메모:
# 논문 1: 
# 논문 2:
# 논문 3:

---
## 3. 바꿔보기: 검색 로직 커스텀

### 수정 포인트

| 항목 | 기본값 | 커스텀 아이디어 |
|------|--------|----------------|
| 검색 필드 | 전체 | 제목만, 초록만 |
| 카테고리 | 전체 | cs.CL, cs.AI, q-bio |
| 날짜 범위 | 전체 | 최근 2년만 |
| 정렬 | relevance | 최신순 |

In [ ]:
# Cell 16: 커스텀 검색 함수 (Before/After)

# Before: 기본 검색
def search_basic(query):
    return search_arxiv_papers(query, max_results=10)

# After: 심리학 특화 검색
def search_psychology_papers(query, max_results=10, years_back=2):
    """
    심리학 연구용 커스텀 검색
    - q-bio.NC (Neurons and Cognition) 카테고리 우선
    - 최근 N년 논문만
    - 제목+초록에서 검색
    """
    from datetime import datetime, timedelta
    
    # 날짜 필터 계산
    cutoff_date = datetime.now() - timedelta(days=years_back*365)
    
    # arXiv 쿼리 문법: cat:카테고리 AND (ti:제목 OR abs:초록)
    enhanced_query = f"(ti:{query} OR abs:{query})"
    
    client = arxiv.Client()
    search = arxiv.Search(
        query=enhanced_query,
        max_results=max_results * 2,  # 필터링 위해 더 많이 검색
        sort_by=arxiv.SortCriterion.SubmittedDate
    )
    
    papers = []
    for result in client.results(search):
        # 날짜 필터
        if result.published.replace(tzinfo=None) < cutoff_date:
            continue
        
        papers.append({
            "title": result.title,
            "authors": [a.name for a in result.authors],
            "abstract": result.summary,
            "published": result.published.strftime("%Y-%m-%d"),
            "categories": result.categories,
            "pdf_url": result.pdf_url
        })
        
        if len(papers) >= max_results:
            break
    
    return papers

# 비교 실행
query = "cognitive load working memory"
print("=== 기본 검색 ===")
basic = search_basic(query)
print(f"결과: {len(basic)}개")

print("\n=== 심리학 특화 검색 (최근 2년) ===")
psych = search_psychology_papers(query, years_back=2)
print(f"결과: {len(psych)}개")

In [ ]:
# DIY Cell 17: 본인만의 필터 추가하기

def my_custom_search(query, max_results=10):
    """
    TODO: 본인 연구에 맞게 커스텀하세요
    
    아이디어:
    - 특정 저자 필터: if 'Author Name' in authors
    - 키워드 필터: if 'keyword' in abstract.lower()
    - 카테고리 필터: if 'cs.CL' in categories
    """
    # 여기에 코드 작성
    pass

# 테스트
# results = my_custom_search("your query")

In [ ]:
# Cell 18: LLM으로 검색 쿼리 자동 생성
from openai import OpenAI

def generate_search_queries(research_topic, num_queries=3):
    """
    연구 주제에서 다양한 검색 쿼리 자동 생성
    """
    client = OpenAI()
    
    prompt = f"""Generate {num_queries} different arXiv search queries for the following research topic.
Each query should focus on a different aspect of the topic.
Return only the queries, one per line.

Research topic: {research_topic}
"""
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    
    queries = response.choices[0].message.content.strip().split('\n')
    return [q.strip() for q in queries if q.strip()]

# 테스트 (API key 있을 때만)
if OPENAI_API_KEY:
    topic = "AI agents for psychological research"
    queries = generate_search_queries(topic)
    print(f"생성된 쿼리:")
    for i, q in enumerate(queries, 1):
        print(f"{i}. {q}")
else:
    print("API key 없음 - 이 기능은 건너뜁니다")

In [ ]:
# DIY Cell 19: 자동 생성된 쿼리로 검색

my_topic = "your research topic"  # <- 수정

if OPENAI_API_KEY:
    # 쿼리 생성
    auto_queries = generate_search_queries(my_topic, num_queries=3)
    
    # 각 쿼리로 검색
    all_results = {}
    for query in auto_queries:
        results = search_arxiv_papers(query, max_results=3)
        all_results[query] = results
        print(f"\n🔍 '{query}' → {len(results)}개")
        for p in results:
            print(f"   - {p['title'][:60]}...")

In [ ]:
# Cell 20: 검색 결과 비교 (원본 vs 수정본)
query = "cognitive psychology LLM"

# 기본 검색
basic_results = search_arxiv_papers(query, max_results=5)

# 심리학 특화 검색
psych_results = search_psychology_papers(query, max_results=5, years_back=2)

print("📊 검색 방법 비교\n")
print(f"기본 검색: {len(basic_results)}개")
for p in basic_results[:3]:
    print(f"  - {p['title'][:50]}... ({p['published']})")

print(f"\n심리학 특화 (2년): {len(psych_results)}개")
for p in psych_results[:3]:
    print(f"  - {p['title'][:50]}... ({p['published']})")

---
## 4. 토론

### 토론 질문

1. **검색 결과의 질을 어떻게 평가할 수 있을까?**
   - precision vs recall
   - 관련성 점수?

2. **Agent Laboratory vs PaSa 중 어떤 것을 베이스로 쓸까?**
   - Agent Lab: 문헌 검색 자동화 (워크샵에서는 이 부분만)
   - PaSa: 검색 특화 (PPO 기반 선택)

3. **추가하고 싶은 기능은?**
   - PubMed 통합?
   - 한글 논문 (DBpia, RISS)?
   - 자동 PDF 다운로드?

---
## 5. PaSa 스타일 Crawler + Selector (공식 API만 사용)

PaSa의 핵심 아키텍처를 **Semantic Scholar 공식 API**로 구현합니다.

### 왜 이 방식인가?

| 항목 | 설명 |
|------|------|
| **공식 스택** | Semantic Scholar 자체가 SPECTER2를 사용 (Allen AI 공식) |
| **API 직접 제공** | `embedding.specterv2` 필드로 임베딩 요청 가능 |
| **검증된 방식** | [allenai/s2search](https://github.com/allenai/s2search) 공식 리랭커 |
| **무료** | API key 없이도 사용 가능 (rate limit만 있음) |

참고: [Semantic Scholar API](https://www.semanticscholar.org/product/api) | [SPECTER2 GitHub](https://github.com/allenai/SPECTER2)

In [ ]:
# Cell 21: PaSa 스타일 Agent (Semantic Scholar 공식 API 사용)
import requests
import numpy as np
from typing import List, Dict, Optional

class PaSaStyleAgent:
    """
    PaSa 아키텍처를 Semantic Scholar 공식 API로 구현
    
    - Crawler: Semantic Scholar 검색 API
    - Selector: Semantic Scholar Recommendations API + SPECTER2 임베딩
    
    출처: https://api.semanticscholar.org/api-docs/
    """
    
    def __init__(self, api_key: Optional[str] = None):
        """
        Parameters:
        - api_key: Semantic Scholar API key (선택)
                   없어도 동작하지만 rate limit 제한 (100 req/5분)
                   발급: https://www.semanticscholar.org/product/api
        """
        self.api_key = api_key
        self.base_url = "https://api.semanticscholar.org/graph/v1"
        self.rec_url = "https://api.semanticscholar.org/recommendations/v1"
    
    def _headers(self):
        """API 헤더 생성"""
        headers = {}
        if self.api_key:
            headers['x-api-key'] = self.api_key
        return headers
    
    def crawl(self, query: str, limit: int = 20, year_from: int = None) -> List[Dict]:
        """
        Crawler: 키워드로 논문 검색
        
        PaSa의 Google Search → arXiv 과정을 대체
        """
        url = f"{self.base_url}/paper/search"
        params = {
            "query": query,
            "limit": limit,
            "fields": "paperId,title,authors,year,abstract,citationCount,venue,url"
        }
        if year_from:
            params["year"] = f"{year_from}-"
        
        response = requests.get(url, params=params, headers=self._headers())
        
        if response.status_code == 429:
            print("⚠️ Rate limit 초과 - API key 발급을 권장합니다")
            print("   https://www.semanticscholar.org/product/api")
            return []
        elif response.status_code != 200:
            print(f"⚠️ API 오류: {response.status_code}")
            return []
        
        papers = []
        for p in response.json().get('data', []):
            papers.append({
                'paperId': p.get('paperId'),
                'title': p.get('title', ''),
                'abstract': p.get('abstract', ''),
                'authors': [a.get('name', '') for a in p.get('authors', [])],
                'year': p.get('year'),
                'citationCount': p.get('citationCount', 0),
                'venue': p.get('venue', ''),
                'url': p.get('url', '')
            })
        
        return papers
    
    def get_recommendations(self, paper_ids: List[str], limit: int = 10) -> List[Dict]:
        """
        Selector: 시드 논문 기반 추천 (Semantic Scholar 공식 Recommendations API)
        
        내부적으로 SPECTER2 임베딩 + SVM 사용
        출처: https://api.semanticscholar.org/api-docs/recommendations
        """
        url = f"{self.rec_url}/papers"
        
        payload = {
            "positivePaperIds": paper_ids,
            "negativePaperIds": []
        }
        params = {
            "fields": "paperId,title,authors,year,abstract,citationCount,venue,url",
            "limit": limit
        }
        
        response = requests.post(url, json=payload, params=params, headers=self._headers())
        
        if response.status_code != 200:
            print(f"⚠️ Recommendations API 오류: {response.status_code}")
            return []
        
        papers = []
        for p in response.json().get('recommendedPapers', []):
            papers.append({
                'paperId': p.get('paperId'),
                'title': p.get('title', ''),
                'abstract': p.get('abstract', ''),
                'authors': [a.get('name', '') for a in p.get('authors', [])],
                'year': p.get('year'),
                'citationCount': p.get('citationCount', 0),
                'venue': p.get('venue', ''),
                'url': p.get('url', '')
            })
        
        return papers
    
    def search(self, query: str, limit: int = 20, expand: bool = True, 
               expand_limit: int = 10, year_from: int = None) -> Dict:
        """
        PaSa 스타일 전체 파이프라인: Crawl → Select → Expand
        
        Parameters:
        - query: 검색 주제
        - limit: 초기 검색 결과 수
        - expand: 추천 기반 확장 여부 (PaSa의 citation expansion 대체)
        - expand_limit: 확장 시 추가할 논문 수
        - year_from: 최소 연도
        
        Returns:
        - {"crawled": [...], "recommended": [...], "all": [...]}
        """
        print(f"🔍 Step 1: Crawling '{query}'")
        crawled = self.crawl(query, limit=limit, year_from=year_from)
        print(f"   → {len(crawled)}개 논문 수집")
        
        recommended = []
        if expand and crawled:
            # 상위 3개 논문을 시드로 추천 확장
            seed_ids = [p['paperId'] for p in crawled[:3] if p['paperId']]
            if seed_ids:
                print(f"📊 Step 2: Expanding from {len(seed_ids)} seed papers")
                recommended = self.get_recommendations(seed_ids, limit=expand_limit)
                print(f"   → {len(recommended)}개 추천 논문 추가")
        
        # 중복 제거 후 합치기
        seen_ids = set()
        all_papers = []
        
        for p in crawled + recommended:
            if p['paperId'] and p['paperId'] not in seen_ids:
                seen_ids.add(p['paperId'])
                all_papers.append(p)
        
        print(f"✅ 총 {len(all_papers)}개 논문 (중복 제거됨)")
        
        return {
            "crawled": crawled,
            "recommended": recommended,
            "all": all_papers
        }

print("✅ PaSaStyleAgent 클래스 정의 완료")
print("   - crawl(): 키워드 검색 (Crawler)")
print("   - get_recommendations(): 시드 기반 추천 (Selector)")
print("   - search(): 전체 파이프라인")

In [ ]:
# Cell 22: PaSa 스타일 검색 테스트
# API key 없이도 동작 (rate limit 주의)

agent = PaSaStyleAgent(api_key=SEMANTIC_SCHOLAR_API_KEY if 'SEMANTIC_SCHOLAR_API_KEY' in dir() else None)

# 검색 실행
results = agent.search(
    query="depression detection natural language processing",
    limit=10,           # Crawler: 10개 검색
    expand=True,        # Selector: 추천 확장
    expand_limit=5,     # 추천 5개 추가
    year_from=2022
)

# 결과 출력
print(f"\n{'='*70}")
print("📚 검색 결과")
print(f"{'='*70}\n")

for i, paper in enumerate(results['all'][:10], 1):
    source = "🔍" if paper in results['crawled'] else "📊"
    print(f"{i}. {source} {paper['title']}")
    print(f"   {paper['year']} | 인용 {paper['citationCount']} | {paper['url']}")
    print()

### DIY: 본인 연구 주제로 검색

In [ ]:
# DIY Cell 23: 본인 연구 주제로 검색
my_query = "your research topic"  # <- 수정!

my_results = agent.search(
    query=my_query,
    limit=15,
    expand=True,
    expand_limit=10,
    year_from=2020
)

# 결과를 DataFrame으로 보기
import pandas as pd

df = pd.DataFrame(my_results['all'])
df['source'] = df.apply(lambda x: 'crawled' if x['paperId'] in [p['paperId'] for p in my_results['crawled']] else 'recommended', axis=1)
display(df[['title', 'year', 'citationCount', 'source']].head(15))

### PaSa 원본 vs 우리 구현 비교

| 항목 | PaSa 원본 | 우리 구현 |
|------|-----------|----------|
| **Crawler** | Google Search API (유료) | Semantic Scholar API (무료) |
| **Selector** | 7B LLM fine-tuned (GPU) | SPECTER2 Recommendations (API) |
| **Citation 확장** | arXiv 직접 크롤링 | Recommendations API |
| **학습 방식** | PPO 강화학습 | 사전학습 임베딩 |
| **비용** | GPU + API 비용 | **완전 무료** |
| **공식 여부** | ByteDance 연구 | **Allen AI 공식 API** |

**결론**: Semantic Scholar API는 내부적으로 SPECTER2를 사용하므로, 우리 구현은 **공식 스택을 그대로 활용**하는 것입니다.

---
## 6. 다음 단계

이어서 실습할 내용:
- **Part 3**: LitLLM으로 검색된 논문을 바탕으로 Related Work 초안 생성
- **Part 4**: AgentReview로 작성한 논문에 피드백 받기

### 참고 자료
- [Semantic Scholar API Docs](https://api.semanticscholar.org/api-docs/)
- [Semantic Scholar Recommendations API](https://api.semanticscholar.org/api-docs/recommendations)
- [SPECTER2 GitHub](https://github.com/allenai/SPECTER2)
- [PaSa 원본 (ByteDance)](https://github.com/bytedance/pasa)